In [ ]:
# Import necessary libraries for Neural Network
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Import TensorFlow for Neural Network
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Verify GPU availability
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Read the data file
df = pd.read_csv("https://raw.githubusercontent.com/NumanESchulich/SchulichDataScience/main/Data%20Science%20I%20(MBAN%206110T)/Group%20Assignment/Datasets/Full%20Dataset%20(RAW).csv")

# Reversing some Binary Columns (Changing column names and reversing values for specific columns)
columns_to_reverse = {
    'DiffWalk': 'NoDiffWalk',
    'HighBP': 'NoHighBP',
    'HighChol': 'NoHighChol',
    'HeartDiseaseorAttack': 'NoHeartDiseaseorAttack',
    'Stroke': 'NoStroke',
    'Smoker': 'NoSmoker',
    'HvyAlcoholConsump': 'NoHvyAlcoholConsump'
}

for old_col, new_col in columns_to_reverse.items():
    df[new_col] = 1 - df[old_col]
    df.drop(columns=[old_col], inplace=True)

# Scaling adjustments

# PhysHlth scaling
def phys_ment_hlth_scale(days):
    if 1 <= days <= 6:
        return 1
    elif 7 <= days <= 12:
        return 0.75
    elif 13 <= days <= 18:
        return 0.5
    elif 19 <= days <= 24:
        return 0.25
    elif 25 <= days <= 30:
        return 0
    return days

df['PhysHlth'] = df['PhysHlth'].apply(phys_ment_hlth_scale)

# MentHlth scaling
df['MentHlth'] = df['MentHlth'].apply(phys_ment_hlth_scale)

# Adding the NotObese column
df['NotObese'] = df['BMI'].apply(lambda x: 1 if x < 30 else 0)

# Age scaling
age_scale = {1: 1, 2: 1, 3: 1, 4: 0.75, 5: 0.75, 6: 0.75, 7: 0.5, 8: 0.5, 9: 0.5, 10: 0.25, 11: 0, 12: 0, 13: 0}
df['Age'] = df['Age'].map(age_scale)

# Feature Engineered Columns
df['PhysicalCondition'] = (df['GenHlth'] + df['PhysHlth']) / 2
df['NoDisease'] = (df['NoHighBP'] + df['NoHighChol']) / 2
df['Lifestyle'] = (df['NoSmoker'] + df['Fruits']) / 2

# Splitting the dataset into features and target variable
X = df[['Age', 'MentHlth', 'NotObese', 'NoDocbcCost', 'PhysicalCondition', 'NoDisease', 'PhysHlth', 'Lifestyle', 'GenHlth', 'Income', 'Fruits']]
y = df['Diabetes_binary']

# Split off 20% of the data for validation later on
X_temp, X_val, y_temp, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling the data
scaler = StandardScaler()
X_temp = scaler.fit_transform(X_temp)
X_val = scaler.transform(X_val)

# Applying SMOTE to the remaining 80% of the data to oversample the minority class significantly
smote = SMOTE(sampling_strategy=1.0, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_temp, y_temp)

# Splitting the resampled dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Define the Neural Network model
def build_nn_model(input_dim):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_dim=input_dim))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Build the model
input_dim = X_train.shape[1]
nn_model = build_nn_model(input_dim)

# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the Neural Network model
history = nn_model.fit(X_train, y_train, validation_data=(X_val, y_val), 
                       epochs=100, batch_size=32, callbacks=[early_stopping])

# Evaluate the model on the validation set
val_loss, val_accuracy = nn_model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

# Predictions and evaluation metrics
y_val_pred = (nn_model.predict(X_val) > 0.5).astype("int32")

print("Classification Report:")
print(classification_report(y_val, y_val_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_val, y_val_pred))

In [3]:
# Import necessary libraries for Neural Network
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Import TensorFlow for Neural Network
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Verify GPU availability
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Read the data file
df = pd.read_csv("https://raw.githubusercontent.com/NumanESchulich/SchulichDataScience/main/Data%20Science%20I%20(MBAN%206110T)/Group%20Assignment/Datasets/Full%20Dataset%20(RAW).csv")

# Reversing some Binary Columns (Changing column names and reversing values for specific columns)
columns_to_reverse = {
    'DiffWalk': 'NoDiffWalk',
    'HighBP': 'NoHighBP',
    'HighChol': 'NoHighChol',
    'HeartDiseaseorAttack': 'NoHeartDiseaseorAttack',
    'Stroke': 'NoStroke',
    'Smoker': 'NoSmoker',
    'HvyAlcoholConsump': 'NoHvyAlcoholConsump'
}

for old_col, new_col in columns_to_reverse.items():
    df[new_col] = 1 - df[old_col]
    df.drop(columns=[old_col], inplace=True)

# Scaling adjustments

# PhysHlth scaling
def phys_ment_hlth_scale(days):
    if 1 <= days <= 6:
        return 1
    elif 7 <= days <= 12:
        return 0.75
    elif 13 <= days <= 18:
        return 0.5
    elif 19 <= days <= 24:
        return 0.25
    elif 25 <= days <= 30:
        return 0
    return days

df['PhysHlth'] = df['PhysHlth'].apply(phys_ment_hlth_scale)

# MentHlth scaling
df['MentHlth'] = df['MentHlth'].apply(phys_ment_hlth_scale)

# Adding the NotObese column
df['NotObese'] = df['BMI'].apply(lambda x: 1 if x < 30 else 0)

# Age scaling
age_scale = {1: 1, 2: 1, 3: 1, 4: 0.75, 5: 0.75, 6: 0.75, 7: 0.5, 8: 0.5, 9: 0.5, 10: 0.25, 11: 0, 12: 0, 13: 0}
df['Age'] = df['Age'].map(age_scale)

# Feature Engineered Columns
df['PhysicalCondition'] = (df['GenHlth'] + df['PhysHlth']) / 2
df['NoDisease'] = (df['NoHighBP'] + df['NoHighChol']) / 2
df['Lifestyle'] = (df['NoSmoker'] + df['Fruits']) / 2

# Splitting the dataset into features and target variable
X = df[['Age', 'MentHlth', 'NotObese', 'NoDocbcCost', 'PhysicalCondition', 'NoDisease', 'PhysHlth', 'Lifestyle', 'GenHlth', 'Income', 'Fruits']]
y = df['Diabetes_binary']

# Split off 20% of the data for validation later on
X_temp, X_val, y_temp, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling the data
scaler = StandardScaler()
X_temp = scaler.fit_transform(X_temp)
X_val = scaler.transform(X_val)

# Applying SMOTE to the remaining 80% of the data to oversample the minority class significantly
smote = SMOTE(sampling_strategy=1.0, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_temp, y_temp)

# Splitting the resampled dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Define the Neural Network model
def build_nn_model(input_dim):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_dim=input_dim))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Build the model
input_dim = X_train.shape[1]
nn_model = build_nn_model(input_dim)

# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the Neural Network model
history = nn_model.fit(X_train, y_train, validation_data=(X_val, y_val), 
                       epochs=100, batch_size=32, callbacks=[early_stopping])

# Evaluate the model on the validation set
val_loss, val_accuracy = nn_model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

# Predictions and evaluation metrics
y_val_pred = (nn_model.predict(X_val) > 0.5).astype("int32")

print("Classification Report:")
print(classification_report(y_val, y_val_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_val, y_val_pred))


Num GPUs Available:  1
Epoch 1/100
8730/8730 [==============================] - 17s 2ms/step - loss: 0.5263 - accuracy: 0.7370 - val_loss: 0.5244 - val_accuracy: 0.7026
Epoch 2/100
8730/8730 [==============================] - 16s 2ms/step - loss: 0.5220 - accuracy: 0.7397 - val_loss: 0.5318 - val_accuracy: 0.6980
Epoch 3/100
8730/8730 [==============================] - 17s 2ms/step - loss: 0.5208 - accuracy: 0.7399 - val_loss: 0.5351 - val_accuracy: 0.7135
Epoch 4/100
8730/8730 [==============================] - 17s 2ms/step - loss: 0.5204 - accuracy: 0.7403 - val_loss: 0.5169 - val_accuracy: 0.7269
Epoch 5/100
8730/8730 [==============================] - 17s 2ms/step - loss: 0.5198 - accuracy: 0.7408 - val_loss: 0.5237 - val_accuracy: 0.7187
Epoch 6/100
8730/8730 [==============================] - 17s 2ms/step - loss: 0.5195 - accuracy: 0.7404 - val_loss: 0.5429 - val_accuracy: 0.7061
Epoch 7/100
8730/8730 [==============================] - 17s 2ms/step - loss: 0.5190 - accuracy: 0.74

Same Model vs Raw Data

In [4]:
# Import necessary libraries for Neural Network
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Import TensorFlow for Neural Network
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Read the data file
df = pd.read_csv("https://raw.githubusercontent.com/NumanESchulich/SchulichDataScience/main/Data%20Science%20I%20(MBAN%206110T)/Group%20Assignment/Datasets/Full%20Dataset%20(RAW).csv")

# Dropping the ID column
df.drop(columns=['ID'], inplace=True)

# Splitting the dataset into features and target variable
X = df.drop(columns=['Diabetes_binary'])
y = df['Diabetes_binary']

# Split off 20% of the data for validation later on
X_temp, X_val, y_temp, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling the data
scaler = StandardScaler()
X_temp = scaler.fit_transform(X_temp)
X_val = scaler.transform(X_val)

# Applying SMOTE to the remaining 80% of the data to oversample the minority class significantly
smote = SMOTE(sampling_strategy=1.0, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_temp, y_temp)

# Splitting the resampled dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Define the Neural Network model
def build_nn_model(input_dim):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_dim=input_dim))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Build the model
input_dim = X_train.shape[1]
nn_model = build_nn_model(input_dim)

# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the Neural Network model
history = nn_model.fit(X_train, y_train, validation_data=(X_val, y_val), 
                       epochs=100, batch_size=32, callbacks=[early_stopping])

# Evaluate the model on the validation set
val_loss, val_accuracy = nn_model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

# Predictions and evaluation metrics
y_val_pred = (nn_model.predict(X_val) > 0.5).astype("int32")

print("Classification Report:")
print(classification_report(y_val, y_val_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_val, y_val_pred))

Epoch 1/100
8730/8730 [==============================] - 18s 2ms/step - loss: 0.5007 - accuracy: 0.7562 - val_loss: 0.5192 - val_accuracy: 0.7316
Epoch 2/100
8730/8730 [==============================] - 17s 2ms/step - loss: 0.4911 - accuracy: 0.7614 - val_loss: 0.5031 - val_accuracy: 0.7397
Epoch 3/100
8730/8730 [==============================] - 17s 2ms/step - loss: 0.4884 - accuracy: 0.7631 - val_loss: 0.5309 - val_accuracy: 0.7194
Epoch 4/100
8730/8730 [==============================] - 17s 2ms/step - loss: 0.4864 - accuracy: 0.7638 - val_loss: 0.5284 - val_accuracy: 0.7169
Epoch 5/100
8730/8730 [==============================] - 17s 2ms/step - loss: 0.4851 - accuracy: 0.7646 - val_loss: 0.5448 - val_accuracy: 0.7232
Epoch 6/100
8730/8730 [==============================] - 17s 2ms/step - loss: 0.4837 - accuracy: 0.7655 - val_loss: 0.5467 - val_accuracy: 0.7109
Epoch 7/100
1586/1586 [==============================] - 2s 1ms/step - loss: 0.5031 - accuracy: 0.7397
Validation Loss: 0.50

Same model vs Raw Data and no SMOTE

In [5]:
# Import necessary libraries for Neural Network
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score

# Import TensorFlow for Neural Network
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Read the data file
df = pd.read_csv("https://raw.githubusercontent.com/NumanESchulich/SchulichDataScience/main/Data%20Science%20I%20(MBAN%206110T)/Group%20Assignment/Datasets/Full%20Dataset%20(RAW).csv")

# Dropping the ID column
df.drop(columns=['ID'], inplace=True)

# Splitting the dataset into features and target variable
X = df.drop(columns=['Diabetes_binary'])
y = df['Diabetes_binary']

# Split off 20% of the data for validation later on
X_temp, X_val, y_temp, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling the data
scaler = StandardScaler()
X_temp = scaler.fit_transform(X_temp)
X_val = scaler.transform(X_val)

# Splitting the remaining 80% of the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)

# Define the Neural Network model
def build_nn_model(input_dim):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_dim=input_dim))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Build the model
input_dim = X_train.shape[1]
nn_model = build_nn_model(input_dim)

# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the Neural Network model
history = nn_model.fit(X_train, y_train, validation_data=(X_val, y_val), 
                       epochs=100, batch_size=32, callbacks=[early_stopping])

# Evaluate the model on the validation set
val_loss, val_accuracy = nn_model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

# Predictions and evaluation metrics
y_val_pred = (nn_model.predict(X_val) > 0.5).astype("int32")

print("Classification Report:")
print(classification_report(y_val, y_val_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_val, y_val_pred))

# Evaluate the model on the test set
test_loss, test_accuracy = nn_model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Predictions and evaluation metrics for test set
y_test_pred = (nn_model.predict(X_test) > 0.5).astype("int32")

print("Test Classification Report:")
print(classification_report(y_test, y_test_pred))

print("Test Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))

# ROC-AUC score
y_val_proba = nn_model.predict(X_val)
roc_auc = roc_auc_score(y_val, y_val_proba)
print(f"Validation ROC-AUC Score: {roc_auc}")

y_test_proba = nn_model.predict(X_test)
roc_auc_test = roc_auc_score(y_test, y_test_proba)
print(f"Test ROC-AUC Score: {roc_auc_test}")

Epoch 1/100
5074/5074 [==============================] - 11s 2ms/step - loss: 0.3250 - accuracy: 0.8621 - val_loss: 0.3092 - val_accuracy: 0.8669
Epoch 2/100
5074/5074 [==============================] - 11s 2ms/step - loss: 0.3181 - accuracy: 0.8641 - val_loss: 0.3108 - val_accuracy: 0.8680
Epoch 3/100
5074/5074 [==============================] - 10s 2ms/step - loss: 0.3171 - accuracy: 0.8647 - val_loss: 0.3097 - val_accuracy: 0.8668
Epoch 4/100
5074/5074 [==============================] - 11s 2ms/step - loss: 0.3166 - accuracy: 0.8652 - val_loss: 0.3098 - val_accuracy: 0.8674
Epoch 5/100
5074/5074 [==============================] - 11s 2ms/step - loss: 0.3157 - accuracy: 0.8652 - val_loss: 0.3090 - val_accuracy: 0.8657
Epoch 6/100
5074/5074 [==============================] - 11s 2ms/step - loss: 0.3155 - accuracy: 0.8651 - val_loss: 0.3091 - val_accuracy: 0.8671
Epoch 7/100
5074/5074 [==============================] - 11s 2ms/step - loss: 0.3153 - accuracy: 0.8655 - val_loss: 0.3100 -

In [7]:
# Import necessary libraries for Neural Network
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.utils import resample

# Import TensorFlow for Neural Network
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Read the data file
df = pd.read_csv("https://raw.githubusercontent.com/NumanESchulich/SchulichDataScience/main/Data%20Science%20I%20(MBAN%206110T)/Group%20Assignment/Datasets/Full%20Dataset%20(RAW).csv")

# Dropping the ID column
df.drop(columns=['ID'], inplace=True)

# Splitting the dataset into features and target variable
X = df.drop(columns=['Diabetes_binary'])
y = df['Diabetes_binary']

# Split off 20% of the data for the final test set
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Combine the remaining data
df_train_val = pd.concat([X_train_val, y_train_val], axis=1)

# Separate majority and minority classes
df_majority = df_train_val[df_train_val.Diabetes_binary == 0]
df_minority = df_train_val[df_train_val.Diabetes_binary == 1]

# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=len(df_majority),    # to match majority class
                                 random_state=42)  # reproducible results

# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])

# Split the upsampled data into features and target variable
X_upsampled = df_upsampled.drop(columns=['Diabetes_binary'])
y_upsampled = df_upsampled['Diabetes_binary']

# Split the upsampled data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_upsampled, y_upsampled, test_size=0.2, random_state=42)

# Scaling the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Define the Neural Network model
def build_nn_model(input_dim):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_dim=input_dim))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Build the model
input_dim = X_train.shape[1]
nn_model = build_nn_model(input_dim)

# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the Neural Network model
history = nn_model.fit(X_train, y_train, validation_data=(X_val, y_val), 
                       epochs=100, batch_size=32, callbacks=[early_stopping])

# Evaluate the model on the validation set
val_loss, val_accuracy = nn_model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

# Predictions and evaluation metrics on validation set
y_val_pred = (nn_model.predict(X_val) > 0.5).astype("int32")

print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

print("Validation Confusion Matrix:")
print(confusion_matrix(y_val, y_val_pred))

# ROC-AUC score for validation set
y_val_proba = nn_model.predict(X_val)
roc_auc_val = roc_auc_score(y_val, y_val_proba)
print(f"Validation ROC-AUC Score: {roc_auc_val}")

# Evaluate the model on the test set
test_loss, test_accuracy = nn_model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Predictions and evaluation metrics on test set
y_test_pred = (nn_model.predict(X_test) > 0.5).astype("int32")

print("Test Classification Report:")
print(classification_report(y_test, y_test_pred))

print("Test Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))

# ROC-AUC score for test set
y_test_proba = nn_model.predict(X_test)
roc_auc_test = roc_auc_score(y_test, y_test_proba)
print(f"Test ROC-AUC Score: {roc_auc_test}")


Epoch 1/100
8730/8730 [==============================] - 18s 2ms/step - loss: 0.5167 - accuracy: 0.7466 - val_loss: 0.5053 - val_accuracy: 0.7508
Epoch 2/100
8730/8730 [==============================] - 17s 2ms/step - loss: 0.5093 - accuracy: 0.7504 - val_loss: 0.5036 - val_accuracy: 0.7524
Epoch 3/100
8730/8730 [==============================] - 18s 2ms/step - loss: 0.5072 - accuracy: 0.7521 - val_loss: 0.5040 - val_accuracy: 0.7533
Epoch 4/100
8730/8730 [==============================] - 18s 2ms/step - loss: 0.5058 - accuracy: 0.7526 - val_loss: 0.5012 - val_accuracy: 0.7531
Epoch 5/100
8730/8730 [==============================] - 18s 2ms/step - loss: 0.5051 - accuracy: 0.7524 - val_loss: 0.5041 - val_accuracy: 0.7532
Epoch 6/100
8730/8730 [==============================] - 17s 2ms/step - loss: 0.5042 - accuracy: 0.7530 - val_loss: 0.5047 - val_accuracy: 0.7533
Epoch 7/100
8730/8730 [==============================] - 17s 2ms/step - loss: 0.5031 - accuracy: 0.7539 - val_loss: 0.5046 -